In [3]:
from imp import reload
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys

sys.path.append("..")

import d2l_pytorch.d2l as d2l

reload(d2l)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


D:\Temp\ipykernel_14124\4031633957.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [4]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics(
  "F:\python_code\DL\Datasets\JayChouLyrics\jaychou_lyrics.txt"
)

In [5]:
def one_hot(x, n_class, dtype=torch.float32):
  x = x.long()
  res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
  res.scatter_(1, x.view(-1, 1), 1)
  return res


In [6]:
x = torch.tensor([0, 2])
x.long()
# one_hot(x, vocab_size)
F.one_hot(x, vocab_size)


tensor([[1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]])

In [7]:
X = torch.arange(10).view(2, 5)
inputs = d2l.to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)


5 torch.Size([2, 1028])


In [8]:
# 初始化模型参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print("will use", device)


will use cuda


In [9]:
def get_params():
  def _one(shape):
    ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
    return torch.nn.Parameter(ts, requires_grad=True)

  W_xh = _one((num_inputs, num_hiddens))
  W_hh = _one((num_hiddens, num_hiddens))
  b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
  W_hq = _one((num_hiddens, num_outputs))
  b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
  return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])


In [10]:
# 定义模型
def init_rnn_state(batch_size, num_hiddens, device):
  return (torch.zeros((batch_size, num_hiddens), device=device),)


In [11]:
def rnn(inputs, state, params):
  W_xh, W_hh, b_h, W_hq, b_q = params
  (H,) = state
  outputs = []
  for X in inputs:
    H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
    Y = torch.matmul(H, W_hq) + b_q
    outputs.append(Y)

  return outputs, (H,)


In [12]:
from d2l_pytorch.d2l import to_onehot


state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)


5 torch.Size([2, 1028]) torch.Size([2, 256])


In [13]:
# 定义预测函数
def predict_rnn(
  prefix,
  num_chars,
  rnn,
  params,
  init_rnn_state,
  num_hiddens,
  vocab_size,
  device,
  idx_to_char,
  char_to_idx,
):
  state = init_rnn_state(1, num_hiddens, device)
  output = [char_to_idx[prefix[0]]]

  for t in range(num_chars + len(prefix) - 1):
    X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
    (Y, state) = rnn(X, state, params)

    if t < len(prefix) - 1:
      output.append(char_to_idx[prefix[t + 1]])
    else:
      output.append(int(Y[0].argmax(dim=1).item()))

  return "".join([idx_to_char[i] for i in output])


In [16]:
predict_rnn(
  "分开", 10, rnn, params, init_rnn_state, num_hiddens, vocab_size, device, idx_to_char, char_to_idx
)

'分开书年解败赏休及为旁辈'

In [ ]:
def grad_clipping(params, theta, device):
  norm = torch.tensor([0.0], device=device)
  for param in params:
    norm += (param.grad.data**2).sum()

  norm = norm.sqrt().item()
  if norm > theta:
    for param in params:
      param.grad.data *= theta / norm
